# TODO: 
- determine a good example of inputs to use in the example Excel
- improve the instructions in the Excel
- consider moving some functions to src file
- Improve error checking
- Add resilience inputs and outage time selection
- Could add scenario description and save the site + description
- Check inputs against

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import copy
import os
from src.post_and_poll import get_api_results

root_url = "https://developer.nrel.gov/api/reopt/stable"

API_KEY = 'DEMO_KEY'  # REPLACE WITH YOUR API KEY

In [ ]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

In [ ]:
# Specify file name: 
data_file = "multi_site_inputs.xlsx"

##############################################################################################################
site_data =  os.path.join(".", 'multi_site_inputs', data_file)
posts_path = os.path.join(".", 'inputs') # To save posts
outputs_path = os.path.join(".", 'outputs')
rates_path = os.path.join(".", 'electric_rates')
results_summary_path = os.path.join(".", 'results_summary')
##############################################################################################################

In [ ]:
# Get Data
df_site = pd.read_excel(site_data, sheet_name="reopt_inputs", index_col=0) 
df_loads = pd.read_excel(site_data, sheet_name="interval_load", index_col=0)  
df_monthly_loads = pd.read_excel(site_data, sheet_name="monthly_loads", index_col=0)  

## The code below is draft and needs a lot of improvement

In [ ]:
# Create posts Dict by site then scenario
# e.g., posts[site_name]

posts = dict() # Dictionary for all posts by site name
for site in df_site: # iterate over sites
    post = dict()
    # Populate all site-specific inputs from Excel
    for label in df_site[site].keys():
        val = df_site[site][label]
        if not pd.isna(val): # ignore empty entries
            key, subkey = label.split(" | ")
            if key not in post:
                post[key] = {}
            # Check if custom utility rate json is provided
            if subkey == "urdb_response":
                with open(os.path.join(rates_path, val), 'r') as fp:
                    post[key][subkey] = json.load(fp)
            else:
                post[key][subkey] = val

    ## Add load and correct timesteps per hour ## 
    if sum(df_loads[site]) > 0:
        load_profile = df_loads[site].to_list()
        load_profile = [float(v) for v in load_profile] 
        post["ElectricLoad"]["loads_kw"] = load_profile


    # Add post to dict
    posts[site] = copy.deepcopy(post)

    with open(os.path.join(posts_path, site + ".json"), 'w') as fp:
        json.dump(posts[site], fp)

In [ ]:
responses = []

for site in df_site:
    responses.append(
        get_api_results(
            posts[site], 
            results_file=os.path.join(outputs_path, site + '.json'),
            api_url=root_url, 
            API_KEY=API_KEY
        )
    )

### Optionally load saved responses

In [ ]:
responses = []

for site in df_site:
    with open(os.path.join(outputs_path, site + ".json"), 'r') as fp:
        responses.append(json.load(fp))

### Summarizing multiple scenario results

There are two options for making a summary of multiple scenarios' resutls:
1. Write to a csv using a template with column headers for desired summary keys (scalar values only)
2. Write all inputs, outputs, and dispatch to an Excel spreadsheet
^^TBD
3. Use the `generation_results_table` endpoint to create a summary Excel


#### Option 1: Use a template CSV to collect certain results

#### Option 3: Use Results Spreadsheet API endpoint
/job/generate_results_table - takes a list of run_uuid's and creates a results table spreadsheet to download in response

In [ ]:
run_uuids = []
for response in responses:
    run_uuids.append(response["run_uuid"])

In [ ]:
# Construct the URL with run_uuid[] as query parameters
results_url = f"{root_url}/job/generate_results_table/?api_key={API_KEY}&table_config_name=custom_table_webtool"

for i, uuid in enumerate(run_uuids):
    results_url += f"&run_uuid%5B{i}%5D={uuid}"

print(f"Requesting custom comparison table from: {results_url}")

try:
    # Make the GET request to retrieve the final comparison table
    response = requests.get(results_url)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Check the response and save the Excel file
    if response.status_code == 200:
        os.makedirs(results_summary_path, exist_ok=True)
        output_path = os.path.join(results_summary_path, f'{uuid}-results-table.xlsx')
        
        with open(output_path, "wb") as o:
            o.write(response.content)
        print(f"File saved to {output_path}")
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")